In [1]:
import matplotlib.pyplot as plt
from matplotlib import rcParams,font_manager,colors,patches
import numpy as np

In [2]:
#some old stuff, please ignore
#x = x.replace(',w_act,w_pass,delta','').replace('{','').replace('0x','').split(';')
#x = [y.split(',') for y in x]
#F = np.vstack([list("{0:036b}".format(int(p,16))) for p,_,_,_ in x]).astype(int)
#Fempty = np.vstack([list("{0:036b}".format(int(p,16))) for _,p,_,_ in x]).astype(int)
#N = np.array([int(n) for _,_,n,_ in x])
#ind = np.array([int(i) for _,_,_,i in x])

In [3]:
#directory where policy_params.txt is saved
direc = 'C:/Users/svo/Documents/FourinarowData/'

In [4]:
#Loading in policy parameters
x = np.loadtxt(direc + 'policy_params.txt',dtype=int)
F = x[:,:36]
Fempty = x[:,36:72]
N = x[:,72]
ind = x[:,73]
w = np.array([0.8,0.2,3.5,6,0.8,0.2,3.5,6,0.8,0.2,3.5,6,0.8,0.2,3.5,6,0])
w = w[ind]

In [6]:
np.savetxt(direc + 'policy_params.txt',x)

In [ ]:
#fucntion which "plots" a 4-in-a-row board
cm = colors.LinearSegmentedColormap.from_list('gray_red_map', [colors.to_rgb('darkgray'), 
                                                            colors.to_rgb('red')], N=100)    

def display_board(bp,wp,move,color,d=None):
    fig = plt.figure()
    ax = fig.add_subplot(111,aspect='equal')
    ax.vlines(np.arange(-0.5,9.5,1),-0.5,3.5,color='black')
    ax.hlines(np.arange(-0.5,4.5,1),-0.5,8.5,color='black')
    for i in range(36):
        if(bp[i]==1):
            circ = patches.Circle((i%9,i//9),0.33,color="black",fill=True)
            circ = ax.add_patch(circ)
        if(wp[i]==1):
            circ = patches.Circle((i%9,i//9),0.33,color="white",fill=True)
            circ = ax.add_patch(circ)
    if move !=99:
        circ = patches.Circle((move%9,move//9),0.33,color=color,fill=False)
        circ = ax.add_patch(circ)
    if not d is None:
        plt.imshow(np.reshape(d,[4,9]), cmap=cm,interpolation='nearest',origin='lower',vmin=0,vmax=0.5)
    else:
        rec = patches.Rectangle((-0.5,-0.5),9,4,color="gray",fill=True,zorder=1)
        ax.add_patch(rec)
    
    ax.axis('off')
    plt.show()

In [ ]:
#example board position
bp = np.array([ 0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0 ])
wp = np.array([ 0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0 ])

In [ ]:
def compute_policy(bp,wp):
    active_features_black = np.logical_and(np.sum(F*bp,axis=1)==np.sum(F,axis=1),
                                           np.sum(Fempty*(bp+wp),axis=1)==0).astype(int)
    active_features_white = np.logical_and(np.sum(F*wp,axis=1)==np.sum(F,axis=1),
                                           np.sum(Fempty*(bp+wp),axis=1)==0).astype(int)
    almost_active_features_black = np.logical_and(np.sum(F*wp,axis=1)==0,
                                                  np.logical_and(np.sum(F*bp,axis=1)==(np.sum(F,axis=1)-1),
                                                                 np.sum(Fempty*(bp+wp),axis=1)==0)).astype(int)
    V = np.sum(Fempty*(active_features_white - active_features_black)[:,None]*w[:,None],axis=0)
    V += np.sum(F*(bp==0)[None,:]*almost_active_features_black[:,None]*w[:,None],axis=0)
    V += 1/np.sqrt((np.repeat(np.arange(4),9)-1.5)**2 + (np.tile(np.arange(9),4)-4)**2)
    V = V-np.max(V)
    return np.exp(V)/np.sum(np.exp(V))


In [ ]:
compute_policy(bp,wp).reshape([4,9])

In [ ]:
display_board(bp,wp,99,'black',d=compute_policy(bp,wp))

In [8]:
np.argwhere(np.zeros(36)==0).flatten()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35], dtype=int64)